In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/product_demand.csv')
df = df.dropna()
df.head()

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
0,Product_0993,Whse_J,Category_028,2012/7/27,100
1,Product_0979,Whse_J,Category_028,2012/1/19,500
2,Product_0979,Whse_J,Category_028,2012/2/3,500
3,Product_0979,Whse_J,Category_028,2012/2/9,500
4,Product_0979,Whse_J,Category_028,2012/3/2,500


In [3]:
product_ids = []
warehouse_ids = []
category_ids = []
years = []
months = []
days = []
demands = []

warehouses = list(df['Warehouse'].unique())

for index, row in df.iterrows():
    product_ids.append(int(row['Product_Code'].split('_')[1]))
    
    warehouse_ids.append(warehouses.index(row['Warehouse']))
    
    category_ids.append(int(row['Product_Category'].split('_')[1]))
    
    ymd = str(row['Date']).split('/')
    years.append(int(ymd[0]))
    months.append(int(ymd[1]))
    days.append(int(ymd[2]))
    
    demands.append(int(row['Order_Demand']))

df.insert(0, 'product_id', product_ids)
df.insert(1, 'warehouse_id', warehouse_ids)
df.insert(2, 'category_id', category_ids)
df.insert(3, 'year', years)
df.insert(4, 'month', months)
df.insert(5, 'day', days)
df.insert(6, 'demand', demands)

df = df.drop(columns=['Product_Code', 'Warehouse', 'Product_Category', 'Date', 'Order_Demand'])
df.head()

ValueError: invalid literal for int() with base 10: '(1)'

In [ ]:
input_columns = df[['product_id', 'warehouse_id', 'category_id', 'year', 'month', 'day']].to_numpy()
output_column = df['demand'].values

In [ ]:
train_input, test_input, train_output, test_output = train_test_split(input_columns, output_column, test_size=.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression().fit(train_input, train_output)
print(model.score(test_input, test_output))

In [5]:
df['Order_Demand'].astype(int)

ValueError: invalid literal for int() with base 10: '(1)'